In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=3):  # Updated to reflect 3 output classes if needed
        super(ImprovedCNN, self).__init__()

        # Convolutional block 1
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(5, 15), stride=(2, 2), padding=(2, 2)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Convolutional block 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(3, 7), stride=(2, 2), padding=(1, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Convolutional block 3
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 5), stride=(1, 1), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2)
        )

        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(6656, 256)  # Adjusted based on output dimensions
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

        # Dropout layer
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
      # Convolutional layers
      x = self.conv1(x)
      x = self.conv2(x)
      x = self.conv3(x)

      # Flatten the features
      x = self.flatten(x)

      # Fully connected layers with dropout
      x = F.relu(self.fc1(x))
      x = self.dropout(x)
      x = F.relu(self.fc2(x))
      x = self.dropout(x)
      x = self.fc3(x)

      return x



# Instantiate the improved model
model = ImprovedCNN(num_classes=3)  # Updated to reflect the potential multi-label task
print(model)

ImprovedCNN(
  (conv1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 15), stride=(2, 2), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 7), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=6656, out_features=256,

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import numpy as np
import torch.optim as optim
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/CNN_Test')

from EEGEyeNet import EEGEyeNetDataset


In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [6]:
# Define paths
data_file_path = '/content/drive/MyDrive/Colab Notebooks/Position_task_with_dots_synchronised_min.npz'

# Create dataset instance
train_dataset = EEGEyeNetDataset(data_file_path, transpose=True)

# Split dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Instantiate the model
output_size = 2  # Assuming output size is known and matches the label dimensions
model = ImprovedCNN(num_classes=output_size)
model = model.to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # For regression tasks
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

# Training loop
epochs = 15
print("Starting training...")
for epoch in range(epochs):
    print(f"Epoch {epoch+1} started...")
    model.train()
    running_loss = 0.0
    for inputs, labels, _ in train_loader:  # Third return value is index, not used
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validation step
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels, _ in val_loader:
            inputs, labels = inputs.to('cuda' if torch.cuda.is_available() else 'cpu'), labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f'Validation Loss: {val_loss/len(val_loader):.4f}')

print('Training completed.')

# Save the trained model
torch.save(model.state_dict(), '/content/drive/My Drive/Colab Notebooks/CNN_Test/trained_model.pth')



loading data...
[[  1.  408.1 315.1]
 [  1.  640.7 519.1]
 [  1.  404.2 118.8]
 ...
 [177.  115.5 306.1]
 [177.  732.  310.3]
 [177.  632.2 353.6]]
Starting training...
Epoch 1 started...
Epoch [1/15], Loss: 45992.2378
Validation Loss: 26308.3894
Epoch 2 started...
Epoch [2/15], Loss: 27459.6587
Validation Loss: 19152.6550
Epoch 3 started...
Epoch [3/15], Loss: 23382.1772
Validation Loss: 20760.6573
Epoch 4 started...
Epoch [4/15], Loss: 22549.6369
Validation Loss: 20446.4031
Epoch 5 started...
Epoch [5/15], Loss: 21758.5322
Validation Loss: 18337.4681
Epoch 6 started...
Epoch [6/15], Loss: 21227.5086
Validation Loss: 22000.7029
Epoch 7 started...
Epoch [7/15], Loss: 21049.0194
Validation Loss: 18387.2213
Epoch 8 started...
Epoch [8/15], Loss: 20605.7098
Validation Loss: 17912.7505
Epoch 9 started...
Epoch [9/15], Loss: 20221.2188
Validation Loss: 21573.0363
Epoch 10 started...
Epoch [10/15], Loss: 20067.8631
Validation Loss: 18018.0656
Epoch 11 started...
Epoch [11/15], Loss: 19756.68

In [ ]:
print("CUDA available:", torch.cuda.is_available())

CUDA available: True
